In [11]:
import numpy as np
import pandas as pd
import json
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import matplotlib.pyplot as plt
plt.style.use("bmh")
plt.rcParams['figure.figsize'] = [15, 10]
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tools.eval_measures import rmse
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LassoCV, Lasso
import statsmodels.api as sm

import itertools

In [19]:
train = pd.read_csv('datasets/podzielone/ekonometryczne/stacjonarne/train_ekon.csv', index_col=0)
val1 = pd.read_csv('datasets/podzielone/ekonometryczne/stacjonarne/valid1_ekon.csv', index_col=0)
val2 = pd.read_csv('datasets/podzielone/ekonometryczne/stacjonarne/valid2_ekon.csv', index_col=0)
val3 = pd.read_csv('datasets/podzielone/ekonometryczne/stacjonarne/valid3_ekon.csv', index_col=0)
test = pd.read_csv('datasets/podzielone/ekonometryczne/stacjonarne/test_ekon.csv', index_col=0)

train.index = pd.to_datetime(train.index)
val1.index = pd.to_datetime(val1.index)
val2.index = pd.to_datetime(val2.index)
val3.index = pd.to_datetime(val3.index)
test.index = pd.to_datetime(test.index)

whole = pd.concat([train, val1, val2, val3, test]).fillna(method='bfill')

train1 = train.asfreq('B', method='bfill').fillna(method='bfill')
train2 = pd.concat([train, val1]).asfreq('B', method='bfill').fillna(method='bfill')
train3 = pd.concat([train, val1, val2]).asfreq('B', method='bfill').fillna(method='bfill')

val1 = val1.asfreq('B', method='bfill').fillna(method='bfill')
val2 = val2.asfreq('B', method='bfill').fillna(method='bfill')
val3 = val3.asfreq('B', method='bfill').fillna(method='bfill')
test = test.asfreq('B', method='bfill').fillna(method='bfill')

val = pd.concat([val1, val2, val3]).asfreq('B', method='bfill').fillna(method='bfill')


In [20]:
train1["day_of_week"] = train1.index.dayofweek
train1["day_of_year"] = train1.index.dayofweek
train1["week"] = train1.index.week
train1["quarter"] = train1.index.quarter


val["day_of_week"] = val.index.dayofweek
val["day_of_year"] = val.index.dayofweek
val["week"] = val.index.week
val["quarter"] = val.index.quarter


In [21]:
val.head()

,y_return_ratio,TensorFlow__diff_1_shift_4,deep learning__diff_1_shift_4,Artificial Intelligence__diff_1_shift_19,Machine Learning__diff_1_shift_4,neural network__diff_1_shift_9,MXNet__diff_1_shift_9,blockchain_news_diff_1_shift_4,GPU_news_diff_1_shift_17,gaming_news_diff_1_shift_2,SandP_diff_1,NasdaqTech_diff_1,ATVI_gaming_diff_1,TTWO_gaming_diff_1,UBSFY_gaming_diff_1,fps_transformed_ffill_diff_1_shift_2,fps_transformed_interpolated_diff_1_shift_2,fps_interpolated_transformed_diff_1_shift_2,PLUS_DM_diff_1,MINUS_DM_diff_1_shift_8,AD_diff_1,OBV_diff_1,MIDPOINT_diff_1,MIDPRICE_diff_1,SAR_diff_1_shift_13,TEMA_diff_1,AVGPRICE_diff_1,MEDPRICE_diff_1,TYPPRICE_diff_1,WCLPRICE_diff_1,LINEARREG_diff_1,TSF_diff_1,ADD_diff_1,MAX_diff_1,MAXINDEX_diff_1,MIN_diff_1,MININDEX_diff_1_shift_19,MULT_diff_1,ATAN_diff_1,CEIL_diff_1,FLOOR_diff_1,LN_diff_1,LOG10_diff_1,SQRT_diff_1,ATVI_gaming_rr,TTWO_gaming_rr,AMD_rr,SandP_rr,NasdaqTech_rr,day_of_week,day_of_year,week,quarter
2018-01-02,0.030233,-0.032258,0.096774,-0.466667,0.129032,-0.161290,-0.451613,0.096774,0.612903,0.322581,-5.440064,-21.635009,0.084999,0.645001,-0.115000,0.0,0.0,0.0,-1.011151,-1.941051,-6.999100e+06,-6999100.0,0.000000,0.000000,-1.752900,-0.311129,-2.034996,-2.284996,-2.823329,-3.092495,-0.463713,-0.557142,-4.569992,0.0,0.0,0.0,0.0,-905.755435,-0.000102,-4.0,-4.0,-0.019955,-0.008666,-0.139483,0.001341,0.005920,-0.012784,-0.002025,-0.005463,1,1,1,1
2018-01-03,0.065814,-0.129032,0.387097,2.129032,0.516129,-0.483871,-1.354839,0.387097,0.612903,0.322581,8.390015,30.989991,0.470000,1.864998,-0.010000,0.0,0.0,0.0,0.101067,-1.802405,8.356997e+06,8890400.0,0.725006,0.000000,-1.647726,0.807774,1.302498,1.019997,2.630000,3.435001,1.436002,1.605387,2.039993,0.0,0.0,0.0,27.0,400.738646,0.000152,6.0,6.0,0.029785,0.012935,0.208708,0.007407,0.017018,0.026379,0.003129,0.007868,2,2,1,1
2018-01-04,0.005271,-0.032258,0.096774,0.709677,0.129032,-0.161290,-0.451613,0.096774,0.612903,0.322581,15.685059,53.874999,1.005001,2.220001,0.430000,0.0,0.0,0.0,13.253852,-1.263657,1.721392e+07,22867600.0,6.559997,6.864998,-1.548863,3.104048,11.222500,11.724998,12.189997,12.422497,4.130000,4.527692,23.449997,0.0,0.0,0.0,0.0,4738.624389,0.000310,13.0,13.0,0.063739,0.027681,0.457215,0.015722,0.019918,0.082243,0.005831,0.013572,3,3,1,1
2018-01-05,0.008474,-0.032258,0.096774,0.709677,0.129032,-0.161290,-0.451613,0.096774,0.612903,0.354839,16.194946,44.380005,0.275002,0.794999,0.274999,0.0,0.0,0.0,2.457157,-1.583400,-9.684894e+06,14581700.0,0.705001,2.175003,-1.455931,2.932099,6.517498,6.645004,4.803334,3.882499,3.134284,3.315273,13.290008,0.0,0.0,0.0,0.0,2835.681185,0.000025,1.0,1.0,0.005257,0.002283,0.038368,0.004235,0.006994,0.045769,0.005986,0.011030,4,4,1,1
2018-01-08,0.030641,-0.032258,0.096774,0.709677,0.129032,-0.161290,-0.451613,0.096774,1.838710,-0.612903,15.589965,27.575073,0.419998,1.440006,0.020001,0.0,0.0,0.0,-2.068360,2.829703,6.990298e+06,14503100.0,3.449998,0.509995,-1.368575,2.898735,-0.627499,-1.375000,-0.313334,0.217499,2.857999,2.979779,-2.749999,0.0,0.0,0.0,0.0,-591.691496,0.000039,2.0,2.0,0.008438,0.003665,0.061793,0.006441,0.012580,-0.006193,0.005728,0.006779,0,0,2,1


In [22]:
X_train = train1.drop(columns=['y_return_ratio']).asfreq('B', method='bfill')
y_train = train1['y_return_ratio'].asfreq('B', method='bfill')

X_test = val.drop(columns=['y_return_ratio']).asfreq('B', method='bfill')
y_test = val['y_return_ratio'].asfreq('B', method='bfill')

In [25]:
y_train.head()

2012-07-02   -0.026773
2012-07-03    0.026022
2012-07-04   -0.010145
2012-07-05   -0.010145
2012-07-06   -0.019034
Freq: B, Name: y_return_ratio, dtype: float64

In [27]:
scores = []
variables = []
for alpha in [0.0005*x for x in range(1, 2*10**3)]:
    print(alpha)
    try:
        clf = Lasso(alpha=alpha)
        featureSelection = SelectFromModel(clf)
        featureSelection.fit(X_train, y_train)
        train_variables = X_train.columns[featureSelection.get_support()].tolist()
        print(train_variables)
        if len(train_variables) >= 2 and train_variables not in variables:
            prediction = []
            for i in range(len(y_test)):
                trainX = pd.concat([X_train, X_test.iloc[:i]]).asfreq('B', method='bfill')[train_variables]
                trainY = pd.concat([y_train, y_test.iloc[:i]]).asfreq('B', method='bfill')
                y_hat = clf.fit(trainX, trainY).predict(X_test[train_variables].iloc[i:i+1])
                prediction.append(y_hat[0])
            print(rmse(prediction, y_test.values))
            rmse_value = rmse(prediction, y_test.values)
            scores.append((rmse_value, alpha, train_variables))
            variables.append(train_variables)
    except Exception as e: raise e
scores

0.0005
['SandP_diff_1', 'NasdaqTech_diff_1', 'SAR_diff_1_shift_13', 'MAXINDEX_diff_1', 'MIN_diff_1', 'MININDEX_diff_1_shift_19', 'week']
0.025348405185629735
0.001
['SandP_diff_1', 'NasdaqTech_diff_1', 'MAXINDEX_diff_1', 'MIN_diff_1', 'MININDEX_diff_1_shift_19', 'week']
0.02530672028190653
0.0015
['SandP_diff_1', 'NasdaqTech_diff_1', 'MAXINDEX_diff_1', 'MININDEX_diff_1_shift_19', 'week']
0.025305193620981638
0.002
['SandP_diff_1', 'NasdaqTech_diff_1', 'MININDEX_diff_1_shift_19', 'week']
0.0253162914901935
0.0025
['SandP_diff_1', 'NasdaqTech_diff_1', 'MININDEX_diff_1_shift_19', 'week']
0.003
['SandP_diff_1', 'NasdaqTech_diff_1', 'week']
0.025306110193778097
0.0035
['SandP_diff_1', 'NasdaqTech_diff_1']
0.02530462836271658
0.004
['SandP_diff_1', 'NasdaqTech_diff_1']
0.0045000000000000005
['NasdaqTech_diff_1']
0.005
['NasdaqTech_diff_1']
0.0055
['NasdaqTech_diff_1']
0.006
['NasdaqTech_diff_1']
0.006500000000000001
['NasdaqTech_diff_1']
0.007
['NasdaqTech_diff_1']
0.0075
['NasdaqTech_diff_1

[]
0.3385
[]
0.339
[]
0.3395
[]
0.34
[]
0.3405
[]
0.341
[]
0.3415
[]
0.342
[]
0.3425
[]
0.343
[]
0.3435
[]
0.34400000000000003
[]
0.34450000000000003
[]
0.34500000000000003
[]
0.34550000000000003
[]
0.34600000000000003
[]
0.34650000000000003
[]
0.34700000000000003
[]
0.34750000000000003
[]
0.34800000000000003
[]
0.34850000000000003
[]
0.34900000000000003
[]
0.34950000000000003
[]
0.35000000000000003
[]
0.35050000000000003
[]
0.35100000000000003
[]
0.35150000000000003
[]
0.352
[]
0.3525
[]
0.353
[]
0.3535
[]
0.354
[]
0.3545
[]
0.355
[]
0.3555
[]
0.356
[]
0.3565
[]
0.357
[]
0.3575
[]
0.358
[]
0.3585
[]
0.359
[]
0.3595
[]
0.36
[]
0.3605
[]
0.361
[]
0.3615
[]
0.362
[]
0.3625
[]
0.363
[]
0.3635
[]
0.364
[]
0.3645
[]
0.365
[]
0.3655
[]
0.366
[]
0.3665
[]
0.367
[]
0.3675
[]
0.368
[]
0.3685
[]
0.369
[]
0.3695
[]
0.37
[]
0.3705
[]
0.371
[]
0.3715
[]
0.372
[]
0.3725
[]
0.373
[]
0.3735
[]
0.374
[]
0.3745
[]
0.375
[]
0.3755
[]
0.376
[]
0.3765
[]
0.377
[]
0.3775
[]
0.378
[]
0.3785
[]
0.379
[]
0.379

[]
0.712
[]
0.7125
[]
0.713
[]
0.7135
[]
0.714
[]
0.7145
[]
0.715
[]
0.7155
[]
0.716
[]
0.7165
[]
0.717
[]
0.7175
[]
0.718
[]
0.7185
[]
0.719
[]
0.7195
[]
0.72
[]
0.7205
[]
0.721
[]
0.7215
[]
0.722
[]
0.7225
[]
0.723
[]
0.7235
[]
0.724
[]
0.7245
[]
0.725
[]
0.7255
[]
0.726
[]
0.7265
[]
0.727
[]
0.7275
[]
0.728
[]
0.7285
[]
0.729
[]
0.7295
[]
0.73
[]
0.7305
[]
0.731
[]
0.7315
[]
0.732
[]
0.7325
[]
0.733
[]
0.7335
[]
0.734
[]
0.7345
[]
0.735
[]
0.7355
[]
0.736
[]
0.7365
[]
0.737
[]
0.7375
[]
0.738
[]
0.7385
[]
0.739
[]
0.7395
[]
0.74
[]
0.7405
[]
0.741
[]
0.7415
[]
0.742
[]
0.7425
[]
0.743
[]
0.7435
[]
0.744
[]
0.7445
[]
0.745
[]
0.7455
[]
0.746
[]
0.7465
[]
0.747
[]
0.7475
[]
0.748
[]
0.7485
[]
0.749
[]
0.7495
[]
0.75
[]
0.7505000000000001
[]
0.751
[]
0.7515000000000001
[]
0.752
[]
0.7525000000000001
[]
0.753
[]
0.7535000000000001
[]
0.754
[]
0.7545000000000001
[]
0.755
[]
0.7555000000000001
[]
0.756
[]
0.7565000000000001
[]
0.757
[]
0.7575000000000001
[]
0.758
[]
0.7585000000000001
[]


[(0.025348405185629735,
  0.0005,
  ['SandP_diff_1',
   'NasdaqTech_diff_1',
   'SAR_diff_1_shift_13',
   'MAXINDEX_diff_1',
   'MIN_diff_1',
   'MININDEX_diff_1_shift_19',
   'week']),
 (0.02530672028190653,
  0.001,
  ['SandP_diff_1',
   'NasdaqTech_diff_1',
   'MAXINDEX_diff_1',
   'MIN_diff_1',
   'MININDEX_diff_1_shift_19',
   'week']),
 (0.025305193620981638,
  0.0015,
  ['SandP_diff_1',
   'NasdaqTech_diff_1',
   'MAXINDEX_diff_1',
   'MININDEX_diff_1_shift_19',
   'week']),
 (0.0253162914901935,
  0.002,
  ['SandP_diff_1', 'NasdaqTech_diff_1', 'MININDEX_diff_1_shift_19', 'week']),
 (0.025306110193778097, 0.003, ['SandP_diff_1', 'NasdaqTech_diff_1', 'week']),
 (0.02530462836271658, 0.0035, ['SandP_diff_1', 'NasdaqTech_diff_1'])]

In [28]:
scores.sort(key=lambda x: x[1])

In [29]:
scores

[(0.025348405185629735,
  0.0005,
  ['SandP_diff_1',
   'NasdaqTech_diff_1',
   'SAR_diff_1_shift_13',
   'MAXINDEX_diff_1',
   'MIN_diff_1',
   'MININDEX_diff_1_shift_19',
   'week']),
 (0.02530672028190653,
  0.001,
  ['SandP_diff_1',
   'NasdaqTech_diff_1',
   'MAXINDEX_diff_1',
   'MIN_diff_1',
   'MININDEX_diff_1_shift_19',
   'week']),
 (0.025305193620981638,
  0.0015,
  ['SandP_diff_1',
   'NasdaqTech_diff_1',
   'MAXINDEX_diff_1',
   'MININDEX_diff_1_shift_19',
   'week']),
 (0.0253162914901935,
  0.002,
  ['SandP_diff_1', 'NasdaqTech_diff_1', 'MININDEX_diff_1_shift_19', 'week']),
 (0.025306110193778097, 0.003, ['SandP_diff_1', 'NasdaqTech_diff_1', 'week']),
 (0.02530462836271658, 0.0035, ['SandP_diff_1', 'NasdaqTech_diff_1'])]

In [30]:
candidates = [x[2] for x in scores]

In [31]:
results_of_e = []
for feature_set in candidates:
    trainX = X_train[feature_set]
    for p in range(4):
        for q in range(4):
            try:
                model = SARIMAX(y_train, trainX, order=(p,0,q), trend='c', freq='B')
                results = model.fit()
                prediction = results.forecast(steps=len(y_test), exog=X_test[feature_set])
                rmse_value = rmse(prediction, y_test)
                print(rmse_value)
                display(results.summary())
                results_of_e.append(((p,0,q), feature_set, rmse_value))
            except: pass

/home/michal/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/statespace/representation.py:375: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return matrix[[slice(None)]*(matrix.ndim-1) + [0]]
/home/michal/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:320: FutureWarning: Creating a DatetimeIndex by passing range endpoints is deprecated.  Use `pandas.date_range` instead.
  freq=base_index.freq)


0.025634603054170493


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Statespace Model Results                           
==============================================================================
Dep. Variable:         y_return_ratio   No. Observations:                 1435
Model:                        SARIMAX   Log Likelihood                3454.964
Date:                Sun, 14 Apr 2019   AIC                          -6891.927
Time:                        09:45:14   BIC                          -6844.507
Sample:                    07-02-2012   HQIC                         -6874.222
                         - 12-29-2017                                         
Covariance Type:                  opg                                         
============================================================================================
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
intercept                    0.0004      0.001      0.288      0.774      -0.002       0.003
SandP_diff_1                -0.0001   6.71e-05     -1.603      0.109      -0.000    2.39e-05
NasdaqTech_diff_1         6.933e-05   3.91e-05      1.771      0.077   -7.39e-06       0.000
SAR_diff_1_shift_13         -0.0003      0.000     -1.223      0.221      -0.001       0.000
MAXINDEX_diff_1              0.0001      0.000      0.640      0.522      -0.000       0.000
MIN_diff_1                   0.0012      0.000      3.194      0.001       0.000       0.002
MININDEX_diff_1_shift_19    -0.0002      0.000     -1.323      0.186      -0.001       0.000
week                      4.938e-05   4.11e-05      1.201      0.230   -3.12e-05       0.000
sigma2                       0.0005   6.05e-06     78.482      0.000       0.000       0.000
===================================================================================
Ljung-Box (Q):                       45.70   Jarque-Bera (JB):             55091.12
Prob(Q):                              0.25   Prob(JB):                         0.00
Heteroskedasticity (H):               2.96   Skew:                             2.56
Prob(H) (two-sided):                  0.00   Kurtosis:                        32.92
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

0.025685678281934948


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Statespace Model Results                           
==============================================================================
Dep. Variable:         y_return_ratio   No. Observations:                 1435
Model:               SARIMAX(0, 0, 1)   Log Likelihood                3455.717
Date:                Sun, 14 Apr 2019   AIC                          -6891.433
Time:                        09:45:17   BIC                          -6838.744
Sample:                    07-02-2012   HQIC                         -6871.761
                         - 12-29-2017                                         
Covariance Type:                  opg                                         
============================================================================================
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
intercept                    0.0022      0.001      1.734      0.083      -0.000       0.005
SandP_diff_1             -9.612e-05   6.62e-05     -1.453      0.146      -0.000    3.35e-05
NasdaqTech_diff_1         7.081e-05    4.1e-05      1.728      0.084   -9.51e-06       0.000
SAR_diff_1_shift_13         -0.0003      0.000     -1.242      0.214      -0.001       0.000
MAXINDEX_diff_1              0.0001      0.000      0.579      0.563      -0.000       0.000
MIN_diff_1                   0.0012      0.000      3.423      0.001       0.001       0.002
MININDEX_diff_1_shift_19    -0.0003      0.000     -1.649      0.099      -0.001    4.89e-05
week                     -9.665e-07   4.04e-05     -0.024      0.981   -8.01e-05    7.82e-05
ma.L1                       -0.0178      0.024     -0.746      0.456      -0.065       0.029
sigma2                       0.0005   6.44e-06     73.261      0.000       0.000       0.000
===================================================================================
Ljung-Box (Q):                       46.01   Jarque-Bera (JB):             55186.58
Prob(Q):                              0.24   Prob(JB):                         0.00
Heteroskedasticity (H):               2.96   Skew:                             2.56
Prob(H) (two-sided):                  0.00   Kurtosis:                        32.94
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

...

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Statespace Model Results                           
==============================================================================
Dep. Variable:         y_return_ratio   No. Observations:                 1435
Model:               SARIMAX(3, 0, 1)   Log Likelihood                3453.857
Date:                Sun, 14 Apr 2019   AIC                          -6891.715
Time:                        09:47:36   BIC                          -6849.563
Sample:                    07-02-2012   HQIC                         -6875.977
                         - 12-29-2017                                         
Covariance Type:                  opg                                         
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
intercept             0.0021      0.001      1.499      0.134      -0.001       0.005
SandP_diff_1         -0.0001   6.94e-05     -1.575      0.115      -0.000    2.67e-05
NasdaqTech_diff_1  7.468e-05   4.19e-05      1.784      0.074   -7.38e-06       0.000
ar.L1                -0.0089      0.551     -0.016      0.987      -1.089       1.071
ar.L2                 0.0045      0.028      0.160      0.873      -0.051       0.060
ar.L3                 0.0508      0.022      2.332      0.020       0.008       0.093
ma.L1                -0.0095      0.551     -0.017      0.986      -1.090       1.071
sigma2                0.0005   6.92e-06     68.663      0.000       0.000       0.000
===================================================================================
Ljung-Box (Q):                       43.26   Jarque-Bera (JB):             55842.02
Prob(Q):                              0.33   Prob(JB):                         0.00
Heteroskedasticity (H):               2.92   Skew:                             2.57
Prob(H) (two-sided):                  0.00   Kurtosis:                        33.12
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

0.02523144959673037


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Statespace Model Results                           
==============================================================================
Dep. Variable:         y_return_ratio   No. Observations:                 1435
Model:               SARIMAX(3, 0, 2)   Log Likelihood                3451.630
Date:                Sun, 14 Apr 2019   AIC                          -6885.260
Time:                        09:47:39   BIC                          -6837.840
Sample:                    07-02-2012   HQIC                         -6867.555
                         - 12-29-2017                                         
Covariance Type:                  opg                                         
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
intercept             0.0024      0.001      3.238      0.001       0.001       0.004
SandP_diff_1         -0.0001   6.97e-05     -2.111      0.035      -0.000   -1.05e-05
NasdaqTech_diff_1  6.979e-05   4.18e-05      1.671      0.095   -1.21e-05       0.000
ar.L1                 0.7944      0.041     19.524      0.000       0.715       0.874
ar.L2                -0.9240      0.037    -24.990      0.000      -0.997      -0.852
ar.L3                 0.0305      0.023      1.307      0.191      -0.015       0.076
ma.L1                -0.7890      0.032    -24.286      0.000      -0.853      -0.725
ma.L2                 0.9400      0.028     34.086      0.000       0.886       0.994
sigma2                0.0005   6.36e-06     74.772      0.000       0.000       0.000
===================================================================================
Ljung-Box (Q):                       47.37   Jarque-Bera (JB):             54040.60
Prob(Q):                              0.20   Prob(JB):                         0.00
Heteroskedasticity (H):               2.91   Skew:                             2.54
Prob(H) (two-sided):                  0.00   Kurtosis:                        32.63
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

0.025389764049404642


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Statespace Model Results                           
==============================================================================
Dep. Variable:         y_return_ratio   No. Observations:                 1435
Model:               SARIMAX(3, 0, 3)   Log Likelihood                3454.781
Date:                Sun, 14 Apr 2019   AIC                          -6889.562
Time:                        09:47:41   BIC                          -6836.872
Sample:                    07-02-2012   HQIC                         -6869.889
                         - 12-29-2017                                         
Covariance Type:                  opg                                         
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
intercept             0.0022      0.002      0.908      0.364      -0.003       0.007
SandP_diff_1         -0.0001      7e-05     -1.766      0.077      -0.000    1.35e-05
NasdaqTech_diff_1  8.537e-05    4.2e-05      2.030      0.042    2.96e-06       0.000
ar.L1                 0.2254      0.810      0.278      0.781      -1.361       1.812
ar.L2                -0.2559      0.611     -0.419      0.675      -1.453       0.942
ar.L3                 0.0077      0.690      0.011      0.991      -1.344       1.360
ma.L1                -0.2480      0.808     -0.307      0.759      -1.832       1.336
ma.L2                 0.2619      0.627      0.418      0.676      -0.967       1.491
ma.L3                 0.0363      0.702      0.052      0.959      -1.340       1.412
sigma2                0.0005   7.06e-06     67.194      0.000       0.000       0.000
===================================================================================
Ljung-Box (Q):                       40.79   Jarque-Bera (JB):             54987.59
Prob(Q):                              0.44   Prob(JB):                         0.00
Heteroskedasticity (H):               2.91   Skew:                             2.56
Prob(H) (two-sided):                  0.00   Kurtosis:                        32.89
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [32]:
results_of_e

[((0, 0, 0),
  ['SandP_diff_1',
   'NasdaqTech_diff_1',
   'SAR_diff_1_shift_13',
   'MAXINDEX_diff_1',
   'MIN_diff_1',
   'MININDEX_diff_1_shift_19',
   'week'],
  0.025634603054170493),
 ((0, 0, 1),
  ['SandP_diff_1',
   'NasdaqTech_diff_1',
   'SAR_diff_1_shift_13',
   'MAXINDEX_diff_1',
   'MIN_diff_1',
   'MININDEX_diff_1_shift_19',
   'week'],
  0.025685678281934948),
 ((0, 0, 2),
  ['SandP_diff_1',
   'NasdaqTech_diff_1',
   'SAR_diff_1_shift_13',
   'MAXINDEX_diff_1',
   'MIN_diff_1',
   'MININDEX_diff_1_shift_19',
   'week'],
  0.025647113882457988),
 ((0, 0, 3),
  ['SandP_diff_1',
   'NasdaqTech_diff_1',
   'SAR_diff_1_shift_13',
   'MAXINDEX_diff_1',
   'MIN_diff_1',
   'MININDEX_diff_1_shift_19',
   'week'],
  0.025664879130010615),
 ((1, 0, 0),
  ['SandP_diff_1',
   'NasdaqTech_diff_1',
   'SAR_diff_1_shift_13',
   'MAXINDEX_diff_1',
   'MIN_diff_1',
   'MININDEX_diff_1_shift_19',
   'week'],
  0.025652152167719634),
 ((1, 0, 1),
  ['SandP_diff_1',
   'NasdaqTech_diff_1',

In [33]:
sorted(results_of_e, key=lambda x: x[2])

[((3, 0, 2), ['SandP_diff_1', 'NasdaqTech_diff_1'], 0.02523144959673037),
 ((3, 0, 2),
  ['SandP_diff_1', 'NasdaqTech_diff_1', 'MININDEX_diff_1_shift_19', 'week'],
  0.02523317863223083),
 ((0, 0, 0), ['SandP_diff_1', 'NasdaqTech_diff_1'], 0.025313742175773007),
 ((3, 0, 0),
  ['SandP_diff_1',
   'NasdaqTech_diff_1',
   'MAXINDEX_diff_1',
   'MIN_diff_1',
   'MININDEX_diff_1_shift_19',
   'week'],
  0.025321563505211683),
 ((2, 0, 0),
  ['SandP_diff_1', 'NasdaqTech_diff_1', 'MININDEX_diff_1_shift_19', 'week'],
  0.025326283589886842),
 ((2, 0, 0),
  ['SandP_diff_1', 'NasdaqTech_diff_1', 'week'],
  0.025328291598801875),
 ((1, 0, 2),
  ['SandP_diff_1', 'NasdaqTech_diff_1', 'MININDEX_diff_1_shift_19', 'week'],
  0.025330563793844373),
 ((0, 0, 0),
  ['SandP_diff_1',
   'NasdaqTech_diff_1',
   'MAXINDEX_diff_1',
   'MIN_diff_1',
   'MININDEX_diff_1_shift_19',
   'week'],
  0.0253312522346586),
 ((3, 0, 0),
  ['SandP_diff_1',
   'NasdaqTech_diff_1',
   'MAXINDEX_diff_1',
   'MININDEX_diff_

In [34]:
good = [
    ((3, 0, 2),
  ['SandP_diff_1',
   'NasdaqTech_diff_1',
   'SAR_diff_1_shift_13',
   'MAXINDEX_diff_1',
   'MIN_diff_1',
   'MININDEX_diff_1_shift_19',
   'week'],
  0.025648996331490433),
    ((3, 0, 2),
  ['SandP_diff_1',
   'NasdaqTech_diff_1',
   'MAXINDEX_diff_1',
   'MIN_diff_1',
   'MININDEX_diff_1_shift_19',
   'week'],
  0.025411927556820417),
    ((3, 0, 2),
  ['SandP_diff_1',
   'NasdaqTech_diff_1',
   'MAXINDEX_diff_1',
   'MININDEX_diff_1_shift_19',
   'week'],
  0.025360105112704978),
    ((3, 0, 2),
  ['SandP_diff_1', 'NasdaqTech_diff_1', 'MININDEX_diff_1_shift_19', 'week'],
  0.02523317863223083),
    ((3, 0, 2), ['SandP_diff_1', 'NasdaqTech_diff_1'], 0.02523144959673037),
]

In [35]:
sorted(good, key=lambda x: x[2])

[((3, 0, 2), ['SandP_diff_1', 'NasdaqTech_diff_1'], 0.02523144959673037),
 ((3, 0, 2),
  ['SandP_diff_1', 'NasdaqTech_diff_1', 'MININDEX_diff_1_shift_19', 'week'],
  0.02523317863223083),
 ((3, 0, 2),
  ['SandP_diff_1',
   'NasdaqTech_diff_1',
   'MAXINDEX_diff_1',
   'MININDEX_diff_1_shift_19',
   'week'],
  0.025360105112704978),
 ((3, 0, 2),
  ['SandP_diff_1',
   'NasdaqTech_diff_1',
   'MAXINDEX_diff_1',
   'MIN_diff_1',
   'MININDEX_diff_1_shift_19',
   'week'],
  0.025411927556820417),
 ((3, 0, 2),
  ['SandP_diff_1',
   'NasdaqTech_diff_1',
   'SAR_diff_1_shift_13',
   'MAXINDEX_diff_1',
   'MIN_diff_1',
   'MININDEX_diff_1_shift_19',
   'week'],
  0.025648996331490433)]

In [52]:
feature_set = ['SandP_diff_1',
   'MIN_diff_1',
   'MININDEX_diff_1_shift_19']
trainX = X_train[feature_set]
model = SARIMAX(y_train, trainX, order=(3, 0, 2), trend='c', freq='B')
results = model.fit()
prediction = results.forecast(steps=len(y_test), exog=X_test[feature_set])
rmse_value = rmse(prediction, y_test)
print(rmse_value)
display(results.summary())

/home/michal/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


0.025075378256165723


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Statespace Model Results                           
==============================================================================
Dep. Variable:         y_return_ratio   No. Observations:                 1435
Model:               SARIMAX(3, 0, 2)   Log Likelihood                3452.358
Date:                Sun, 14 Apr 2019   AIC                          -6884.716
Time:                        10:00:16   BIC                          -6832.027
Sample:                    07-02-2012   HQIC                         -6865.044
                         - 12-29-2017                                         
Covariance Type:                  opg                                         
============================================================================================
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
intercept                    0.0023      0.001      3.143      0.002       0.001       0.004
SandP_diff_1              -5.41e-05   5.59e-05     -0.967      0.333      -0.000    5.55e-05
MIN_diff_1                   0.0009      0.000      2.342      0.019       0.000       0.002
MININDEX_diff_1_shift_19    -0.0002      0.000     -1.160      0.246      -0.000       0.000
ar.L1                        0.8737      0.047     18.519      0.000       0.781       0.966
ar.L2                       -0.9206      0.046    -19.828      0.000      -1.012      -0.830
ar.L3                        0.0357      0.023      1.536      0.125      -0.010       0.081
ma.L1                       -0.8495      0.041    -20.492      0.000      -0.931      -0.768
ma.L2                        0.9303      0.035     26.875      0.000       0.862       0.998
sigma2                       0.0005   6.59e-06     72.156      0.000       0.000       0.000
===================================================================================
Ljung-Box (Q):                       46.24   Jarque-Bera (JB):             54967.90
Prob(Q):                              0.23   Prob(JB):                         0.00
Heteroskedasticity (H):               2.94   Skew:                             2.57
Prob(H) (two-sided):                  0.00   Kurtosis:                        32.88
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [53]:
last_candidates = [
    ['MIN_diff_1', 'MININDEX_diff_1_shift_19'],
    ['SandP_diff_1', 'NasdaqTech_diff_1'],
    ['SandP_diff_1', 'NasdaqTech_diff_1', 'MININDEX_diff_1_shift_19', 'week'],
]